In [1]:
#load lib
import numpy as np
import re, sys, glob
import itertools
from collections import Counter
from keras.preprocessing import text
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from nltk import ngrams
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import keras
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from keras.models import load_model
import math
import scipy
_EPOCHS=50


Using TensorFlow backend.


In [2]:
#load data

'''
Features order in the data look like this 
print STDOUT "S:".$uttranceInfo{"id"}."\t".$wordcount." ".$graphemecount." ".$uttranceInfo{"total_frames"}.
      " ".$uttranceInfo{"avg_loglike"}." ".$uttranceInfo{"total_AMloglike"}." ".
	  $uttranceInfo{"total_LMloglike"}." ".$uttranceInfo{"duration"}." ".
	  $uttranceInfo{"GER"}."\t".$uttranceInfo{"sentence"}."\t".$uttranceInfo{"grapheme"}.
	  "\t".$uttranceInfo{"WER"}."\n";
'''

train="data/dev.mgb2/wer.feats"
test1="data/eval.mgb2/wer.feats"
test2="data/summa_test20170317/wer.feats"


In [3]:
def load_file (file,type='glass'):
    f=open(file,"r")
    lines=f.readlines()
    _id=[]
    _text=[]
    _numerical=[]
    _char=[]
    _ref=[]
    for x in lines:
        _id.append(x.split('\t')[0])
        _text.append(x.split('\t')[2])
        _char.append(x.split('\t')[3])
        _ref.append(x.split('\t')[4])
        if type == 'glass': 
            _numerical.append(x.split('\t')[1])
        else:
            blac_box_numerical=x.split('\t')[1].split(' ')
            numerical2=' '.join(blac_box_numerical[0:2]+blac_box_numerical[6:])
            _numerical.append(numerical2)
        
    f.close()
    return _id, _numerical, _text, _char, _ref

def word_grams(words, min=1, max=4):
    s = []
    for n in range(min, max):
        for ngram in ngrams(words, n):
            s.append(' '.join(str(i) for i in ngram))
    return s

def return_word_char_list (_text,_char):
    _unigram_word_list = [] 
    _trigram_char_list = []
    for i,j in enumerate(_char):
        
        _char_3gram = word_grams (_char[i].split(),1,3)
        for tri in _char_3gram: 
             if tri not in _trigram_char_list: _trigram_char_list.append(tri)
    
        _words_1gram = word_grams (_text[i].split(),1,2)
        for uni in _words_1gram: 
             if uni not in _unigram_word_list: _unigram_word_list.append(uni)
    return _unigram_word_list, _trigram_char_list 
                    
def vectorize (words,chars,word_list,char_list):
    feature_vector = []
    index=0
    
    temp_words = [0] * len(word_list)
    for word in word_grams (words.split(),1,2): 
        for index, word2 in enumerate(word_list): 
            if word == word2:
                #print (index, word2)
                temp_words[index]+=1
                break
    
    temp_chars = [0] * len(char_list)
    for _char in word_grams (chars.split(),1,3): 
        for index, _char2 in enumerate(char_list): 
            if _char == _char2:
                #print (index, _char2)
                temp_chars[index]+=1
                break            
    return temp_words+temp_chars

def make_features (numerical, text, char, word_list, char_list ):
    allfeats= []
    for i, _id in enumerate (text):
        feat = vectorize (text[i],char[i],word_list,char_list)
        allfeats.append (numerical[i].split()+feat)
        if i % 1000 == 0: print ("Processing: ", i)
    nn=np.array(allfeats)
    #nn_minmax = preprocessing.scale(nn)
       
    print (nn.shape)
    return (nn)
    
def get_wc_err (file,dump):
    print ("Processing: ", file)
    f=open(file,"r")
    lines=f.readlines()
    _wc=[]
    _err=[]
    _real_wc =[]
    _real_err =[]
    for x in lines:
        err = int(x.split('\t')[4].split(' ')[1])
        wc  = int(x.split('\t')[4].split(' ')[2])
        _real_wc.append (wc)    
        _real_err.append (err)
        
        if wc > 20: wc =20
        if err > 20: err =20
        wc = wc - 1 # for whatever reason keras like to start counting from zer
        
        _wc.append (wc)    
        _err.append (err)
    
    unique, counts = np.unique(_wc, return_counts=True)
    print ("wc:unique: ",unique)
    print ("wc:counts: ",counts)
    
    unique, counts = np.unique(_err, return_counts=True)
    print ("err:unique: ",unique)
    print ("err:counts: ",counts)
    
    one_hot_labels_wc = to_categorical(np.array(_wc), num_classes=20)
    one_hot_labels_err = to_categorical(np.array(_err), num_classes=21)
        
    np.savetxt(dump+'.err', np.array(_real_err), fmt='%d')
    np.savetxt(dump+'.wc', np.array(_real_wc), fmt='%d')
    
    print (one_hot_labels_wc.shape,one_hot_labels_err.shape)
    return one_hot_labels_wc, one_hot_labels_err


In [4]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

    

def report (test_vec,test_labels,test,ext):
    
    prob_out='prob.out.'+test+'.'+ext
    pred_out='pred.out.'+test+'.'+ext
    class_out='class.out.'+test+'.'+ext
    
    _shift=0
    if ext == "wc" : _shift=2
    #load models
    model = load_model('./my_model.h5')
    
    #save predictions
    pred = model.predict(test_vec)
    np.savetxt(pred_out, pred, delimiter=' ')    
    
    prob = np.copy(pred)
    i=0
    for row in prob: 
      raw = [sigmoid(i) for i in row]
      norm = [float(i)/sum(raw) for i in raw]
      prob[int(i)]=norm
      i=i+1
    
    #save prob
    np.savetxt(prob_out, prob, delimiter=' ',fmt='%f') 
    
    
    test_classes = np.argmax(prob, axis=1)+_shift
    #save classes
    np.savetxt(class_out, test_classes, delimiter=' ',fmt='%d')
    

    test_labels = np.argmax(test_labels, axis=1) # to remove the Keras to_categorical
    
    

    #print confusion matrix 
    confusion_matrix(test_labels, test_classes)

    #print classification report
    print(classification_report(test_labels, test_classes))


    #print prec, recall and f1
    print ("Precision:\t{:0.3f}".format(precision_score(test_labels, test_classes,average='weighted')))
    print ("Recall:  \t{:0.3f}".format(recall_score(test_labels, test_classes,average='weighted')))
    print ("F1 Score:\t{:0.3f}".format(f1_score(test_labels, test_classes,average='weighted')))
    print ("Accuracy:\t{:0.3f}".format(accuracy_score(test_labels, test_classes)))
    


In [5]:
id, numerical, text, char, ref =load_file (train,'glass')

word_list, char_list = return_word_char_list (text, char)
numerical_len=len(numerical[0].split())

feat_len=(len(word_list+char_list)+numerical_len)
print ("Feature dimension: ", feat_len)



train_feats = make_features (numerical, text, char, word_list, char_list )
train_wc, train_err = get_wc_err (train,'train')



id, numerical, text, char, ref =load_file (test1,'glass')
test1_feats = make_features (numerical, text, char, word_list, char_list )
test1_wc, test1_err = get_wc_err (test1,'test1')


id, numerical, text, char, ref =load_file (test2,'glass')
test2_feats = make_features (numerical, text, char, word_list, char_list )
test2_wc, test2_err = get_wc_err (test2,'test2')



Feature dimension:  14964
Processing:  0
Processing:  1000
Processing:  2000
Processing:  3000
Processing:  4000
Processing:  5000
(5842, 14964)
Processing:  data/dev.mgb2/wer.feats
wc:unique:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
wc:counts:  [  2   7  33 101 166 237 326 372 459 455 446 454 429 390 362 283 290 228
 184 618]
err:unique:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
err:counts:  [661 776 740 612 565 438 333 265 228 193 127 138 123  90  77  70  73  50
  43  34 206]
(5842, 20) (5842, 21)
Processing:  0
Processing:  1000
Processing:  2000
Processing:  3000
Processing:  4000
Processing:  5000
(5655, 14964)
Processing:  data/eval.mgb2/wer.feats
wc:unique:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
wc:counts:  [  1   9  21  76 128 219 305 413 462 509 519 509 481 435 363 328 235 185
 125 332]
err:unique:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
err:counts:  [619 886 868 800 588 498 358 217

In [6]:
#def dnn (feats,labels):
#estimate glass-box word count (make sure to add one to your estimation)
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=feat_len))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(20, activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_feats,train_wc, epochs=_EPOCHS,batch_size=32)
model.save('./my_model.h5', overwrite=True)

report (test1_feats,test1_wc,'test1.glass','wc')
report (test2_feats,test2_wc,'test2.glass','wc')


Epoch 1/50
5842/5842 [==============================] - 77s - loss: 2.9640 - acc: 0.1275    
Epoch 2/50
5842/5842 [==============================] - 57s - loss: 2.5190 - acc: 0.1618    
Epoch 3/50
5842/5842 [==============================] - 62s - loss: 2.3495 - acc: 0.1953    
Epoch 4/50
5842/5842 [==============================] - 56s - loss: 2.2295 - acc: 0.2129    
Epoch 5/50
5842/5842 [==============================] - 49s - loss: 2.1296 - acc: 0.2289    
Epoch 6/50
5842/5842 [==============================] - 49s - loss: 2.0563 - acc: 0.2393    
Epoch 7/50
5842/5842 [==============================] - 43s - loss: 1.9967 - acc: 0.2641    
Epoch 8/50
5842/5842 [==============================] - 42s - loss: 1.9570 - acc: 0.2645    
Epoch 9/50
5842/5842 [==============================] - 52s - loss: 1.8958 - acc: 0.2814    
Epoch 10/50
5842/5842 [==============================] - 47s - loss: 1.8929 - acc: 0.2843    
Epoch 11/50
5842/5842 [==============================] - 45s - loss: 

C:\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classific

             precision    recall  f1-score   support

          0       0.00      0.00      0.00         1
          1       0.00      0.00      0.00         9
          2       0.00      0.00      0.00        21
          3       0.00      0.00      0.00        76
          4       0.06      0.01      0.01       128
          5       0.02      0.00      0.01       219
          6       0.01      0.00      0.01       305
          7       0.05      0.02      0.03       413
          8       0.03      0.02      0.02       462
          9       0.06      0.02      0.03       509
         10       0.05      0.09      0.07       519
         11       0.08      0.09      0.09       509
         12       0.06      0.03      0.04       481
         13       0.06      0.09      0.07       435
         14       0.06      0.02      0.03       363
         15       0.04      0.09      0.06       328
         16       0.03      0.03      0.03       235
         17       0.05      0.10      0.07   

In [7]:
#def dnn (feats,labels):
#estimate glass-box error count
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=feat_len))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(21, activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_feats,train_err, epochs=_EPOCHS,batch_size=32)
model.save('./my_model.h5', overwrite=True)

report (test1_feats,test1_err,'test1.glass','err')
report (test2_feats,test2_err,'test2.glass','err')

Epoch 1/50
5842/5842 [==============================] - 58s - loss: 2.7997 - acc: 0.1498    
Epoch 2/50
5842/5842 [==============================] - 48s - loss: 2.3674 - acc: 0.1879    
Epoch 3/50
5842/5842 [==============================] - 41s - loss: 2.2580 - acc: 0.1926    
Epoch 4/50
5842/5842 [==============================] - 42s - loss: 2.1873 - acc: 0.2138    
Epoch 5/50
5842/5842 [==============================] - 42s - loss: 2.1386 - acc: 0.2224    
Epoch 6/50
5842/5842 [==============================] - 42s - loss: 2.1115 - acc: 0.2379    
Epoch 7/50
5842/5842 [==============================] - 43s - loss: 2.0702 - acc: 0.2424    
Epoch 8/50
5842/5842 [==============================] - 41s - loss: 2.0474 - acc: 0.2424    
Epoch 9/50
5842/5842 [==============================] - 41s - loss: 2.0216 - acc: 0.2523    
Epoch 10/50
5842/5842 [==============================] - 41s - loss: 2.0031 - acc: 0.2552    
Epoch 11/50
5842/5842 [==============================] - 41s - loss: 

C:\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.52      0.09      0.15       619
          1       0.34      0.14      0.20       886
          2       0.21      0.30      0.25       868
          3       0.21      0.05      0.08       800
          4       0.15      0.59      0.24       588
          5       0.18      0.07      0.10       498
          6       0.17      0.16      0.17       358
          7       0.13      0.23      0.16       217
          8       0.17      0.04      0.07       208
          9       0.14      0.13      0.13       160
         10       0.18      0.03      0.05       106
         11       0.16      0.15      0.15        89
         12       0.10      0.25      0.15        60
         13       0.38      0.08      0.13        39
         14       0.00      0.00      0.00        31
         15       0.00      0.00      0.00        29
         16       0.10      0.28      0.15        18
         17       0.09      0.06      0.07   

In [8]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt


def MAE (txt,glass,black,ref): 
    MAE_glass=mean_absolute_error(glass, ref)
    MAE_black=mean_absolute_error(black, ref)

    RMS_glass = sqrt(mean_squared_error(ref, glass))
    RMS_black = sqrt(mean_squared_error(ref, black))
    
    personWER_glass = scipy.stats.pearsonr(glass, ref)
    personWER_black = scipy.stats.pearsonr(black, ref)
    
    print  ('MAE:', txt,  "%.2f" % MAE_glass,  "%.2f" % MAE_black)
    print  ('RMSE:', txt,  "%.2f" % RMS_glass,  "%.2f" % RMS_black, '\n')
    print ('Person Correlation:', txt, "%.2f" % personWER_glass[0], "%.2f" % personWER_black[0])
    

def parse_results(onetwo):
    one_two = str (onetwo)
    print ("Analysis for: ", one_two)
    #wc per sentence
    glass_file_wc = 'class.out.test'+one_two+'.glass.wc'
    black_file_wc = 'class.out.test'+one_two+'.black.wc'
    ref_file_wc   =  'test'+one_two+'.wc'

    glass_np_wc = np.loadtxt(glass_file_wc,usecols=range(0,1),dtype='float32')+1
    black_np_wc = np.loadtxt(black_file_wc,usecols=range(0,1),dtype='float32')+1
    ref_np_wc   = np.loadtxt(ref_file_wc,usecols=range(0,1),dtype='float32')

    MAE ('glass/black: WC :',glass_np_wc,black_np_wc,ref_np_wc)
    

    #err per sentence
    glass_file_err = 'class.out.test'+one_two+'.glass.err'
    black_file_err = 'class.out.test'+one_two+'.black.err'
    ref_file_err   = 'test'+one_two+'.err'

    glass_np_err = np.loadtxt(glass_file_err,usecols=range(0,1),dtype='float32')
    black_np_err = np.loadtxt(black_file_err,usecols=range(0,1),dtype='float32')
    ref_np_err   = np.loadtxt(ref_file_err,usecols=range(0,1),dtype='float32')


    
    MAE ('glass/black: ERR :',glass_np_err,black_np_err,ref_np_err)
    

    #wer per sentence
    glass_np_wer=np.divide(glass_np_err,glass_np_wc)*100
    black_np_wer=np.divide(black_np_err,black_np_wc)*100
    ref_np_wer=np.divide(ref_np_err,ref_np_wc)*100

    MAE ('glass/black: WER :',glass_np_wer,black_np_err,ref_np_wer)
    
    
    #Overall WER
    wer_ref   = np.sum(ref_np_err)/np.sum(ref_np_wc)*100
    wer_glass = np.sum(glass_np_err)/np.sum(glass_np_wc)*100
    wer_black = np.sum(black_np_err)/np.sum(black_np_wc)*100
    print ('Overall WER: glass/black:', "%.2f" % wer_ref, "%.2f" % wer_glass, "%.2f" % wer_black)
    print ('###########')

    #dump WER for more analysis
    np.savetxt('wer_ref.'+one_two, ref_np_wer,fmt='%.2f') 
    np.savetxt('wer_black.'+one_two, black_np_wer,fmt='%.2f') 
    np.savetxt('wer_glass.'+one_two, glass_np_wer,fmt='%.2f') 




In [9]:
##Process the black-box features:
id, numerical, text, char, ref =load_file (train,'black')

word_list, char_list = return_word_char_list (text, char)
numerical_len=len(numerical[0].split())

feat_len=(len(word_list+char_list)+numerical_len)
print ("Feature dimension: ", feat_len)



train_feats = make_features (numerical, text, char, word_list, char_list )
train_wc, train_err = get_wc_err (train,'train')



id, numerical, text, char, ref =load_file (test1,'black')
test1_feats = make_features (numerical, text, char, word_list, char_list )
test1_wc, test1_err = get_wc_err (test1,'test1')


id, numerical, text, char, ref =load_file (test2,'black')
test2_feats = make_features (numerical, text, char, word_list, char_list )
test2_wc, test2_err = get_wc_err (test2,'test2')



Feature dimension:  14960
Processing:  0
Processing:  1000
Processing:  2000
Processing:  3000
Processing:  4000
Processing:  5000
(5842, 14960)
Processing:  data/dev.mgb2/wer.feats
wc:unique:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
wc:counts:  [  2   7  33 101 166 237 326 372 459 455 446 454 429 390 362 283 290 228
 184 618]
err:unique:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
err:counts:  [661 776 740 612 565 438 333 265 228 193 127 138 123  90  77  70  73  50
  43  34 206]
(5842, 20) (5842, 21)
Processing:  0
Processing:  1000
Processing:  2000
Processing:  3000
Processing:  4000
Processing:  5000
(5655, 14960)
Processing:  data/eval.mgb2/wer.feats
wc:unique:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
wc:counts:  [  1   9  21  76 128 219 305 413 462 509 519 509 481 435 363 328 235 185
 125 332]
err:unique:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
err:counts:  [619 886 868 800 588 498 358 217

In [10]:
#def dnn (feats,labels):
#estimate glass-box word count (make sure to add one to your estimation)
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=feat_len))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(20, activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_feats,train_wc, epochs=_EPOCHS,batch_size=32)
model.save('./my_model.h5', overwrite=True)

report (test1_feats,test1_wc,'test1.black','wc')
report (test2_feats,test2_wc,'test2.black','wc')

Epoch 1/50
5842/5842 [==============================] - 58s - loss: 2.5935 - acc: 0.1679    
Epoch 2/50
5842/5842 [==============================] - 42s - loss: 2.1472 - acc: 0.2412    
Epoch 3/50
5842/5842 [==============================] - 40s - loss: 1.9480 - acc: 0.2845    
Epoch 4/50
5842/5842 [==============================] - 40s - loss: 1.8373 - acc: 0.3073    
Epoch 5/50
5842/5842 [==============================] - 41s - loss: 1.7319 - acc: 0.3468    
Epoch 6/50
5842/5842 [==============================] - 41s - loss: 1.6501 - acc: 0.3733    
Epoch 7/50
5842/5842 [==============================] - 41s - loss: 1.5681 - acc: 0.3990    
Epoch 8/50
5842/5842 [==============================] - 41s - loss: 1.4958 - acc: 0.4399    
Epoch 9/50
5842/5842 [==============================] - 41s - loss: 1.4224 - acc: 0.4743    
Epoch 10/50
5842/5842 [==============================] - 42s - loss: 1.3517 - acc: 0.5118    
Epoch 11/50
5842/5842 [==============================] - 41s - loss: 

C:\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classific

             precision    recall  f1-score   support

          0       0.00      0.00      0.00         1
          1       0.00      0.00      0.00         9
          2       0.00      0.00      0.00        21
          3       0.00      0.00      0.00        76
          4       0.04      0.01      0.01       128
          5       0.16      0.03      0.05       219
          6       0.15      0.12      0.14       305
          7       0.14      0.03      0.04       413
          8       0.16      0.15      0.15       462
          9       0.14      0.12      0.13       509
         10       0.13      0.10      0.11       519
         11       0.16      0.13      0.14       509
         12       0.09      0.07      0.08       481
         13       0.10      0.15      0.12       435
         14       0.11      0.09      0.10       363
         15       0.07      0.12      0.09       328
         16       0.06      0.04      0.05       235
         17       0.04      0.08      0.05   

In [11]:
#def dnn (feats,labels):
#estimate black-box error count
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=feat_len))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(21, activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_feats,train_err, epochs=_EPOCHS,batch_size=32)
model.save('./my_model.h5', overwrite=True)

report (test1_feats,test1_err,'test1.black','err')
report (test2_feats,test2_err,'test2.black','err')

Epoch 1/50
5842/5842 [==============================] - 55s - loss: 2.5923 - acc: 0.1556    
Epoch 2/50
5842/5842 [==============================] - 41s - loss: 2.2768 - acc: 0.1982    
Epoch 3/50
5842/5842 [==============================] - 41s - loss: 2.1464 - acc: 0.2333    
Epoch 4/50
5842/5842 [==============================] - 41s - loss: 2.0539 - acc: 0.2573    
Epoch 5/50
5842/5842 [==============================] - 43s - loss: 1.9636 - acc: 0.2963    
Epoch 6/50
5842/5842 [==============================] - 42s - loss: 1.8794 - acc: 0.3247    
Epoch 7/50
5842/5842 [==============================] - 42s - loss: 1.7854 - acc: 0.3596    
Epoch 8/50
5842/5842 [==============================] - 41s - loss: 1.6903 - acc: 0.4000    
Epoch 9/50
5842/5842 [==============================] - 42s - loss: 1.6092 - acc: 0.4305    
Epoch 10/50
5842/5842 [==============================] - 42s - loss: 1.5192 - acc: 0.4658    
Epoch 11/50
5842/5842 [==============================] - 42s - loss: 

C:\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
#summary
parse_results (1)
print ('\n')
parse_results (2)


Analysis for:  1
MAE: glass/black: WC : 2.67 2.51
RMSE: glass/black: WC : 3.08 3.06 

Person Correlation: glass/black: WC : 0.91 0.88
MAE: glass/black: ERR : 1.71 2.12
RMSE: glass/black: ERR : 2.27 2.82 

Person Correlation: glass/black: ERR : 0.82 0.72
MAE: glass/black: WER : 13.70 28.24
RMSE: glass/black: WER : 17.76 35.94 

Person Correlation: glass/black: WER : 0.70 0.55
Overall WER: glass/black: 33.03 30.32 29.80
###########


Analysis for:  2
MAE: glass/black: WC : 2.55 2.67
RMSE: glass/black: WC : 3.00 3.25 

Person Correlation: glass/black: WC : 0.89 0.84
MAE: glass/black: ERR : 1.82 2.31
RMSE: glass/black: ERR : 2.43 3.11 

Person Correlation: glass/black: ERR : 0.86 0.78
MAE: glass/black: WER : 12.73 24.31
RMSE: glass/black: WER : 17.05 34.55 

Person Correlation: glass/black: WER : 0.80 0.66
Overall WER: glass/black: 28.51 26.45 28.65
###########
